In [ ]:
def ner(f,f1,a1):
    #loading the required libraries
    from simpletransformers.ner import NERModel
    import pandas as pd
    import logging
    import re
    from nltk.tokenize import TreebankWordTokenizer, sent_tokenize
    from transformers import pipeline
    from transformers import AutoTokenizer, AutoModelForTokenClassification
    import openpyxl
    from openpyxl import load_workbook
    logging.basicConfig(level=logging.DEBUG)
    transformers_logger = logging.getLogger('transformers')
    transformers_logger.setLevel(logging.WARNING)

    
   #reading  text from input files
    file="/host/Sahithi/myenv/input/input/"+f1
    with open(file, "r") as cur:
        text=cur.read()

    #preprocessing of data
    exceptions = {
            r'\d+mg': r'\g<0> |\g<1>',
            r'\d+:\d+': r'\g<0>',
            r'[\,\-\(\)]': r' \g<0> ',
            r'(?<!\d)\d+\.\d+(?!\d)': r'\g<0>',
            r'\d+/\d+': r'\g<0>',
            r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}': r'\g<0>',
            r'[A-Za-z]\d': r'\g<0>',
            r'[^\.\s]\.[^\.\s]': r'\g<0>',
            r'\d+\.\d+':r'\g<0>',
        }
    tokenizer = TreebankWordTokenizer()
    tokens = tokenizer.tokenize(text)
    processed_tokens = []

    a = ["COVID-19", "4/5Unable", "workstation:<ID", "patientÂ\x92s"]
    # List to store processed sentences
    sentences = [] 

    for token in tokens:
        if token in a:
            processed_tokens.append(token)
        elif token not in exceptions and (":" in token or "," in token or "/" in token or "." in token or "<" in token or ">" in token or "-" in token):
            if re.match(r'\d+/\d+', token) or re.match(r'\d+:\d+', token):
                processed_tokens.append(token)
            elif re.match(r'(?<!\d)\d+\.\d+(?!\d)', token):
                processed_tokens.append(token)
            else:
                split_tokens = re.split(r'(:|,|/|\.|<|>|-)', token)
                processed_tokens.extend([t for t in split_tokens if t])
        elif token in exceptions:
            processed_tokens.append(exceptions[token])
        else:
            processed_tokens.append(token)

    # Iterate over sentences and append processed tokens
    current_sentence =""
    sentences=""
    for token in processed_tokens:
        current_sentence=current_sentence+" "+token
   
        if token.endswith('.') or token.endswith('!') or token.endswith('?'):
            sentences=sentences+" "+current_sentence
            current_sentence=""

    # Append the remaining tokens as the last sentence
    if current_sentence:
        sentences=sentences+current_sentence
    c=[]
    sentences1=sentences.replace('REMITTING MULTIPLE SCLEROSIS','remitting multiple sclerosis')
    
    #splitting the total sentences with maximum length as 350(it can be userdefined )
    def process_input(s, max_length):
        if len(s) <= max_length:
            c.append(s)
        else:
            while len(s) > 0:
                chunk = s[:max_length] 
                c.append(chunk)
                s = s[max_length:]


    max_length = 350
    process_input(sentences1, max_length)



    # Annotated file for each document
    a2="/host/Sahithi/myenv/output_entity/"+a1
    df = pd.read_csv(a2, delimiter='\n', header=None, quoting=3)
    annotated_text = df[0].tolist() 

    # making a list 'a'  with the 
    a = []
    for i in annotated_text:
        b = i.split(' ')
        a.append(b)     

    tokenizer = AutoTokenizer.from_pretrained("alvaroalon2/biobert_diseases_ner")
    model = AutoModelForTokenClassification.from_pretrained("alvaroalon2/biobert_diseases_ner")
    pipe = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")
    x=[]
    p=[]

    #c is the chunks of the data with max limit 350
    for i in c: 
        p.extend(pipe(i))
    
    #combining predictions dictioneries of the chunks into a list
    predicted_tokens = []
    for item in p:
        for word in item['word'].split():
            predicted_tokens.append([word,item['entity_group']])

    #making list of annotated_tokens and labels
    annotated_tokens = [[item[0],item[3]] for item in a] 

    #counting the number of annotated tokens of the corresponding conll files
    annotated_num=0
    for i in annotated_tokens:
        if(i[1]!='O'):
            annotated_num+=1 

    #calculating the paramenters required for measuring performance
    tp = []
    tn = []
    fp = []
    fn = []

    for item in predicted_tokens:
        count = 0
        for items in annotated_tokens:
            if item[0] == items[0]  or items[0] in item[0].upper(): 
                count = 1
                if (item[1] == '0') & (items[1]=='O'):
                    tn.append([item,items])
                    annotated_tokens.remove(items)
                    break
                elif (item[1] == 'DISEASE') & (items[1]!='O'):
                    tp.append([item,items])
                    annotated_tokens.remove(items)
                    break
                elif (item[1] == 'DISEASE') & (items[1]=='O'):
                    fp.append([item,items])
                    annotated_tokens.remove(items)
                    break
                elif (item[1] == '0') & (items[1]!='O'):
                    fn.append([item,items])
                    annotated_tokens.remove(items)
                    break

        if count == 0:
            if item[1] == 'DISEASE':
                fp.append([[item,[item[0],'O']]])
            elif item[1] == '0':
                tn.append([[item,[item[0],'O']]])
    print(len(tp),len(fp),len(tn),len(fn))
    
    inscope=0
    flag=False
    anno=0
    if(len(tp)==0 or len(fp)==0 or len(tn)==0 ):
        print("somevalue is zer0")
          
    else:
         
        flag=True

        total=[]
        [total.extend(l) for l in (tp,tn,fp,fn)]

        token=[]
        predict=[]
        annot=[]
        for i in total:
            if(len(i)==2):
                token.append(i[0][0])
                predict.append(i[0][1])
                annot.append(i[1][1])
            elif(len(i)==1):
                for j in i:
                    token.append(j[0][0])
                    predict.append(j[0][1])
                    annot.append(j[1][1])

        output=pd.DataFrame({
                        'token':token,
                        'predictions':predict,
                        'annotated':annot,
                    })
        
    #finding the number of inscope tokens 
        for i in total:  
            if(len(i)==2):
                if(i[0][1]!='0' and i[1][1]!='O'):
                    inscope=inscope+1
                    print(i)
            elif len(i)==1:
                for j in i:
                    if(j[0][1]!='0' and j[1][1]!='O'):
                        inscope=inscope+1
                        print(j)
       
                        
        print("inscope:",inscope)
        
            
        
        
    return len(token),annotated_num,inscope,flag,len(tp),len(tn),len(fp),len(fn)

    

import os
files = os.listdir('/host/Sahithi/myenv/input/input/')
an_files=os.listdir('/host/Sahithi/myenv/output_entity/')
tokens=0
annotated_tokens=0
num_file=0
inscope_tokens=0
tp=0
tn=0
fp=0
fn=0
for an_file in an_files:
    f=os.path.splitext(os.path.basename(an_file))[0]
    for file in files:
        f1=os.path.splitext(os.path.basename(file))[0]
        if f1 in f:
    #calling the function by passing the input file and its corresponding annotated conll file
            t,an,ins,flag,t1,t2,t3,t4=ner(f1,file,an_file)
            if flag==True:
                tokens=tokens+t
                annotated_tokens=annotated_tokens+an
                tp=tp+t1
                tn=tn+t2
                fp=fp+t3
                fn=fn+t4
                inscope_tokens+=ins
                num_file+=1
                
                
            
#finding the performance metrics for overall data          
precision = tp/(tp+fp)
recall = tp/(tp+fn)
f1 = 2*(precision*recall)/(precision+recall)  



In [ ]:
print("number of  tokens:",tokens  )
print("number of annotated tokens:",annotated_tokens)
print("inscope _tokens:",inscope_tokens)
print("true positives:",tp)
print("true negatives:",tn)
print("false positives:",fp)
print("false negatives:",fn)


print("precision:",precision*100)
print("recall:",recall*100)
print("f1 score:",f1*100)